# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import json
import azureml.core
from azureml.core.model import Model
from azureml.core import Workspace, Experiment, Model, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
import os
import pandas as pd
import numpy as np
import json
import requests
import joblib
from sklearn.metrics import confusion_matrix
import itertools

print("SDK version:", azureml.core.VERSION)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'auto-ml-exp'

experiment=Experiment(ws, experiment_name)

In [ ]:
aml_compute_cluster = "heartFailure-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=aml_compute_cluster)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, aml_compute_cluster, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
print(compute_target.get_status().serialize())

In [ ]:

key = "HeartFailure-Dataset"

dataset = ws.datasets[key]

df = dataset.to_pandas_dataframe()
df.describe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 3,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                             compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT", 
                             enable_early_stopping= True,
                             **automl_settings
)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

In [ ]:
print('Best Run Id: ', best_run.id)
print('\n Accuracy: ', best_run_metrics['Accuracy'])
print('\n Metrics: ', best_run_metrics)
print('\n Best Model: ', fitted_model)

In [ ]:
print(fitted_model._final_estimator)

In [ ]:
#TODO: Save the best model
automl_best_model = best_run.register_model(model_name = "best-automl-model", model_path = './outputs/model.pkl', properties={'Accuracy': best_run_metrics['accuracy']})
print(automl_best_model)

In [ ]:
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for props_name in model.properties:
        props = model.properties[props_name]
        print ('\t',props_name, ':', props)
    print('\n')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
os.makedirs('./atoml-model', exist_ok=True)

best_run.download_file('/outputs/model.pkl',os.path.join('./atoml-model','best-automl-model.pkl'))

for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./atoml-model', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)

In [ ]:

model=best_run.register_model(
            model_name = 'best-automl-model', 
            model_path = './outputs/model.pkl'
)

In [ ]:
# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoreScript.py')

# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

In [ ]:

# Create an inference config 
inference_config = InferenceConfig(entry_script='score.py',
                                    environment=best_run.get_environment())

# Deploy the model as a web service
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled=True, enable_app_insights=True)
webservice = Model.deploy(ws, "myservice", [automl_best_model], inference_config, aci_config)
webservice.wait_for_deployment(show_output = True)


In [ ]:
webservice

In [ ]:
print(webservice.state)

print(webservice.scoring_uri)

print(webservice.swagger_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
#test data

test_df = df.sample(5) 
label_df = test_df.pop('DEATH_EVENT')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

In [ ]:
output = webservice.run(test_sample)
print(output)

In [ ]:
# Send a request to the web service you deployed to test it.

import requests

headers = {'Content-type': 'application/json'}

response = requests.post(webservice.scoring_uri, test_sample, headers=headers)

In [ ]:
print(response.text)

In [ ]:
print(label_df)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(webservice.get_logs())

In [ ]:
webservice.delete()